<a href="https://colab.research.google.com/github/pdpavandesai/MergingDiabetesData/blob/main/MergedDatasetPrep.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import warnings
import numpy as np
import pandas as pd
import io
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import f_oneway, chi2_contingency
from sklearn.ensemble import RandomForestRegressor
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingRegressor,RandomForestRegressor,VotingRegressor
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
warnings.filterwarnings('ignore')

In [3]:
from google.colab import files
uploaded = files.upload()

Saving FineTuned_MergedDatasetForDiabetes.csv to FineTuned_MergedDatasetForDiabetes.csv


In [4]:
mg_DS_Actual = pd.read_csv(io.BytesIO(uploaded['FineTuned_MergedDatasetForDiabetes.csv']))
mg_DS_Actual_Copy = mg_DS_Actual.copy()

In [4]:
mg_DS_Actual_Copy.head(15)

,Age,BMI,Gender,HighChol,HighBP,Smoker,HvyAlcoholConsump,HeartDisease,HbA1cLevel,BloodGlucoseLevel,Diabetes
0,9,40.0,0,1.0,1,1,0.0,NaN,NaN,NaN,0
1,7,25.0,0,0.0,0,1,0.0,NaN,NaN,NaN,0
2,9,28.0,0,1.0,1,0,0.0,NaN,NaN,NaN,0
3,11,27.0,0,0.0,1,0,0.0,NaN,NaN,NaN,0
4,11,24.0,0,1.0,1,0,0.0,NaN,NaN,NaN,0
5,10,25.0,1,1.0,1,1,0.0,NaN,NaN,NaN,0
6,9,30.0,0,0.0,1,1,0.0,NaN,NaN,NaN,0
7,11,25.0,0,1.0,1,1,0.0,NaN,NaN,NaN,0
8,9,30.0,0,1.0,1,1,0.0,NaN,NaN,NaN,2
9,8,24.0,1,0.0,0,0,0.0,NaN,NaN,NaN,0


In [ ]:
print('==========Info==========')
print(mg_DS_Actual_Copy.info())
print('')
print('==========Check for number of null Values==========')
print(mg_DS_Actual_Copy.isnull().sum())
print('')
print('==========Finding no.of Unique values in each column==========')
print(mg_DS_Actual_Copy.nunique())
print('')
print('==========Description of dataset==========')
print(mg_DS_Actual_Copy.describe())

==========Info==========
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 282675 entries, 0 to 282674
Data columns (total 11 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   Gender             282675 non-null  int64  
 1   Age                282675 non-null  int64  
 2   HighBP             282675 non-null  int64  
 3   BMI                282675 non-null  float64
 4   Smoker             282675 non-null  int64  
 5   HeartDisease       52894 non-null   float64
 6   HbA1cLevel         52894 non-null   float64
 7   BloodGlucoseLevel  52894 non-null   float64
 8   HighChol           229781 non-null  float64
 9   HvyAlcoholConsump  229781 non-null  float64
 10  Diabetes           282675 non-null  int64  
dtypes: float64(6), int64(5)
memory usage: 23.7 MB
None

==========Check for number of null Values==========
Gender                    0
Age                       0
HighBP                    0
BMI                       0
Smoker

In [5]:
# mg_DS_Actual_Copy['HighChol'] = mg_DS_Actual_Copy['HighChol'].fillna(mg_DS_Actual_Copy['HighChol'].quantile(0.75))
# mg_DS_Actual_Copy['HeartDisease'] = mg_DS_Actual_Copy['HeartDisease'].fillna(mg_DS_Actual_Copy['HeartDisease'].quantile(0.75))
# mg_DS_Actual_Copy['HvyAlcoholConsump'] = mg_DS_Actual_Copy['HvyAlcoholConsump'].fillna(mg_DS_Actual_Copy['HvyAlcoholConsump'].quantile(0.75))
# mg_DS_Actual_Copy['HbA1cLevel'] = mg_DS_Actual_Copy['HbA1cLevel'].fillna(mg_DS_Actual_Copy['HbA1cLevel'].mean())
# mg_DS_Actual_Copy['BloodGlucoseLevel'] = mg_DS_Actual_Copy['BloodGlucoseLevel'].fillna(mg_DS_Actual_Copy['BloodGlucoseLevel'].mean())
imp_mean = IterativeImputer(estimator=RandomForestRegressor(),random_state=0)
imp_mean.fit(mg_DS_Actual_Copy)
display(pd.DataFrame(imp_mean.transform(mg_DS_Actual_Copy)))
print(mg_DS_Actual_Copy.isnull().sum())

,0,1,2,3,4,5,6,7,8,9,10
0,9.0,40.00,0.0,1.0,1.0,1.0,0.0,1.00,5.727000,125.110000,0.0
1,7.0,25.00,0.0,0.0,0.0,1.0,0.0,0.00,5.866000,168.300000,0.0
2,9.0,28.00,0.0,1.0,1.0,0.0,0.0,0.02,5.920000,88.933333,0.0
3,11.0,27.00,0.0,0.0,1.0,0.0,0.0,0.00,5.686833,157.205000,0.0
4,11.0,24.00,0.0,1.0,1.0,0.0,0.0,1.00,5.898000,136.810000,0.0
...,...,...,...,...,...,...,...,...,...,...,...
282670,2.0,34.34,0.0,0.0,0.0,2.0,0.0,0.00,6.500000,160.000000,0.0
282671,5.0,40.69,0.0,0.0,0.0,2.0,1.0,0.00,3.500000,155.000000,0.0
282672,10.0,27.83,1.0,0.0,0.0,1.0,0.0,0.00,5.700000,155.000000,0.0
282673,1.0,35.42,0.0,1.0,0.0,2.0,1.0,0.00,4.000000,100.000000,0.0


Age                       0
BMI                       0
Gender                    0
HighChol              52894
HighBP                    0
Smoker                    0
HvyAlcoholConsump     52894
HeartDisease         229781
HbA1cLevel           229781
BloodGlucoseLevel    229781
Diabetes                  0
dtype: int64


In [6]:
mg_DS_Actual_Copy.head(15)

,Age,BMI,Gender,HighChol,HighBP,Smoker,HvyAlcoholConsump,HeartDisease,HbA1cLevel,BloodGlucoseLevel,Diabetes
0,9,40.0,0,1.0,1,1,0.0,NaN,NaN,NaN,0
1,7,25.0,0,0.0,0,1,0.0,NaN,NaN,NaN,0
2,9,28.0,0,1.0,1,0,0.0,NaN,NaN,NaN,0
3,11,27.0,0,0.0,1,0,0.0,NaN,NaN,NaN,0
4,11,24.0,0,1.0,1,0,0.0,NaN,NaN,NaN,0
5,10,25.0,1,1.0,1,1,0.0,NaN,NaN,NaN,0
6,9,30.0,0,0.0,1,1,0.0,NaN,NaN,NaN,0
7,11,25.0,0,1.0,1,1,0.0,NaN,NaN,NaN,0
8,9,30.0,0,1.0,1,1,0.0,NaN,NaN,NaN,2
9,8,24.0,1,0.0,0,0,0.0,NaN,NaN,NaN,0


In [7]:
fmds_train = mg_DS_Actual_Copy.copy()

In [8]:
y = fmds_train.Diabetes
X = fmds_train.drop(['Diabetes'], axis=1)
print(X)
print(y)

        Age    BMI  Gender  HighChol  HighBP  Smoker  HvyAlcoholConsump  \
0         9  40.00       0       1.0       1       1                0.0   
1         7  25.00       0       0.0       0       1                0.0   
2         9  28.00       0       1.0       1       0                0.0   
3        11  27.00       0       0.0       1       0                0.0   
4        11  24.00       0       1.0       1       0                0.0   
...     ...    ...     ...       ...     ...     ...                ...   
282670    2  34.34       0       NaN       0       2                NaN   
282671    5  40.69       0       NaN       0       2                NaN   
282672   10  27.83       1       NaN       0       1                NaN   
282673    1  35.42       0       NaN       0       2                NaN   
282674    8  22.43       0       NaN       0       0                NaN   

        HeartDisease  HbA1cLevel  BloodGlucoseLevel  
0                NaN         NaN             

In [9]:
X_train, X_val, y_train, y_val = train_test_split(X,y,test_size=0.1, random_state=1)
X_train.shape,y_train.shape,X_val.shape,y_val.shape

((254407, 10), (254407,), (28268, 10), (28268,))

In [10]:
GBR = GradientBoostingRegressor(random_state=1, n_estimators=1000, max_depth=10).fit(X_train,y_train)
print(GBR.score(X_train,y_train))

ValueError: ignored

In [ ]:
GBR_pred = GBR.predict(X_val)
print(GBR_pred)
rms = mean_squared_error(y_val,GBR_pred,squared=True)
print(rms)

In [ ]:
plt.figure(figsize=(10,10))
plt.scatter(y_val,GBR_pred,c='crimson')
plt.yscale('log')
plt.xscale('log')

p1 = max(max(GBR_pred), max(y_val))
p2 = min(min(GBR_pred), min(y_val))
plt.plot([p1,p2],[p1,p2],'b-')
plt.xlabel('Actual Values', fontsize=15)
plt.xlabel('Prediction', fontsize=15)
plt.axis('equal')
plt.show()